# Modelo de planificación centralizada usando R
Notas de clase sobre la planificación centralizada de mercados eléctricos

**Juan David Velásquez Henao**   
jdvelasq@unal.edu.co  
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia  

[Licencia](https://github.com/jdvelasq/ETVL-R/blob/master/LICENCIA.txt)  
[Readme](https://github.com/jdvelasq/ETVL-R/blob/master/Readme.md)

**Software utilizado**.

> Este es un documento interactivo escrito como un notebook de [Jupyter](http://jupyter.org), en el cual se presenta un tutorial sobre la extracción, transformación, visualización y carga de datos usando **R** en el contexto de la ciencia de los datos. Los notebooks de Jupyter permiten incoporar simultáneamente código, texto, gráficos y ecuaciones. El código presentado en este notebook puede ejecutarse en los sistemas operativos Linux y OS X.

> Haga click [aquí](https://github.com/jdvelasq/guias-de-instalacion) para obtener instrucciones detalladas sobre como instalar Jupyter en Windows y Mac OS X.

> Haga clic [aquí](http://nbviewer.jupyter.org/github/jdvelasq/metodos-y-modelos/blob/master/02-05-actividad-modelo-PL-embalses-R.ipynb) para ver la última versión de este documento en nbviewer.

> Descargue la última versión de este documento a su disco duro; luego, carguelo y ejecutelo en línea en [Try Jupyter!](https://try.jupyter.org)


#### Contenido

> * [Introducción](#Introducción)
* [Planteamiento del problema](#Planteamiento-del-problema)
* [Formulación del modelo de PL](#Formulación-del-modelo-de-PL)
    * [Definición de variables](#Definición-de-variables)
    * [Tamaño del problema](#Tamaño-del-problema)
    * [Función objetivo](#Función-objetivo)
    * [Restricciones de demanda](#Restricciones-de-demanda)
    * [Continuidad de los volumenes de los embalses](#Continuidad-de-los-volumenes-de-los-embalses)
    * [Generación máxima](#Generación-máxima)
    * [Volumen máximo](#Volumen-máximo)
* [Solucion](#Solucion)

In [78]:
printvec <- function(w) {
    
    m <- w[w != 0]
    n <- names(m)
    
    for(i in 1:length(m)) {
        
        cat(m[i], ' * ', n[i], sep='', end='\n' )
            
    }
    cat('')
}

# Introducción

[Contenido](#Contenido)

En este documento se ejemplifica el desarrollo de un modelo parametrizado para la planificación de la operación de un sistema hidrotérmico usando programación lineal. Aunque se está utilizando el lenguaje R, el plantemiento presentado es similar a como se realizaría en otras herramientas especializadas en este tipo de problemas, tal como GAMS o AMPL.

# Planteamiento del problema

[Contenido](#Contenido)

Se desea realizar el planeamiento de la operación de un sistema hidrotérmico conformado por una planta hidráulica y otra térmica. Usualmente, siempre hay mas de una planta térmica y una hidráulica por lo que la información siempre se almacena en tablas (data.frames). La información es la siguiente:

In [79]:
P = 16  ## Periodo de planificación en trimestres

In [80]:
## Plantas hidráulicas.
##   hay un registro por cada planta hidro
hidros <- data.frame( Vo   = c( 70),   ## Volumen inicial al principio del periodo de la planificación
                      Vf   = c( 70),   ## Volumen final al terminar el periodo de planificación
                      Vmax = c(100),   ## Volumen máximo
                      Qmax = c( 30),   ## Energía máxima generada
                      rho  = c(  1),   ## Factor de conversión
                      stringsAsFactors = False)  


hidros <- read.csv('hidro.csv', stringsAsFactors = FALSE)

H = nrow(hidros)  ## cantidad de plantas hidraulicas

print(hidros)

  Vo Vf Vmax Qmax rho
1 70 70  100   30   1


In [81]:
aportes <- data.frame( planta  = rep(1, 16),
                       t       = 1:16,
                       aportes = c( 28, 22, 19, 25,  # Aportes hidrológicos
                                    21, 16, 18, 23,
                                    27, 22, 23, 33,
                                    26, 19, 22, 27),
                       stringsAsFactors = False)

print(aportes)

   planta  t aportes
1       1  1      28
2       1  2      22
3       1  3      19
4       1  4      25
5       1  5      21
6       1  6      16
7       1  7      18
8       1  8      23
9       1  9      27
10      1 10      22
11      1 11      23
12      1 12      33
13      1 13      26
14      1 14      19
15      1 15      22
16      1 16      27


In [82]:
## Plantas térmicas.
##   hay un registro por cada planta térmica.
termos <- data.frame( GTmax = c(31),  # Generación máxima
                      CC    = c(15),  # Costo de combustible
                      stringsAsFactors = False)

T <- nrow(termos)  ## cantidad de plantas térmicas

print(termos)

  GTmax CC
1    31 15


In [83]:
## Costo de racionamiento: 
CR <- 1000 # para todas las etapas.

In [84]:
##  Demanda por trimestre: 
dem <- c( 50, 50, 50, 50,  # año 1
          60, 60, 60, 60,  # año 2
          70, 70, 70, 70,  # año 3
          80, 80, 80, 80)  # año 4

# Formulación del modelo de PL

[Contenido](#Contenido)

In [85]:
## carga de la libreria
library(lpSolveAPI)

## Definición de variables

[Contenido](#Contenido)

Se crean los nombres de las variables del problema para facilitar su manejo.

In [88]:
nombres <- c()

In [89]:
create.matrix <- function(prefix, n, m) {
    x <- matrix('', n, m)
    for(i in 1:n) 
        for(j in 1:m)
            x[i,j] <- paste(prefix, '[', i, ',', j, ']', sep='')

    return(x)
}

In [90]:
create.vector <- function(prefix, n) {
    x <- rep('', n)
    for(i in 1:n)
        x[i] <- paste(prefix, '[', i, ']', sep = '')
    return(x)
}

In [91]:
## plantas térmicas

var.gt <- create.matrix('gt',          # generacion termica 
                        nrow(termos),  # numero de plantas termicas
                        P)             # numero de periodos

In [92]:
## plantas hidraulicas
var.vol <- create.matrix( 'vol',          # volumen almacenado
                           nrow(hidros),  # número de plantas hidraulicas
                           P)             # número de periodos

var.q   <- create.matrix( 'q',            # volumen turbinado
                           nrow(hidros),
                           P)

var.ver <- create.matrix( 'ver',          # volumen vertido
                           nrow(hidros),
                           P)

In [93]:
## deficit

#
# genere los nombres de la variable de racionamiento
#
var.def <- create.vector('def', P)

In [94]:
## crea un solo vector con los nombres todas las variables
var.names <- c(c(var.gt), c(var.vol), c(var.q), c(var.ver), var.def)
print(var.names)


 [1] "gt[1,1]"   "gt[1,2]"   "gt[1,3]"   "gt[1,4]"   "gt[1,5]"   "gt[1,6]"  
 [7] "gt[1,7]"   "gt[1,8]"   "gt[1,9]"   "gt[1,10]"  "gt[1,11]"  "gt[1,12]" 
[13] "gt[1,13]"  "gt[1,14]"  "gt[1,15]"  "gt[1,16]"  "vol[1,1]"  "vol[1,2]" 
[19] "vol[1,3]"  "vol[1,4]"  "vol[1,5]"  "vol[1,6]"  "vol[1,7]"  "vol[1,8]" 
[25] "vol[1,9]"  "vol[1,10]" "vol[1,11]" "vol[1,12]" "vol[1,13]" "vol[1,14]"
[31] "vol[1,15]" "vol[1,16]" "q[1,1]"    "q[1,2]"    "q[1,3]"    "q[1,4]"   
[37] "q[1,5]"    "q[1,6]"    "q[1,7]"    "q[1,8]"    "q[1,9]"    "q[1,10]"  
[43] "q[1,11]"   "q[1,12]"   "q[1,13]"   "q[1,14]"   "q[1,15]"   "q[1,16]"  
[49] "ver[1,1]"  "ver[1,2]"  "ver[1,3]"  "ver[1,4]"  "ver[1,5]"  "ver[1,6]" 
[55] "ver[1,7]"  "ver[1,8]"  "ver[1,9]"  "ver[1,10]" "ver[1,11]" "ver[1,12]"
[61] "ver[1,13]" "ver[1,14]" "ver[1,15]" "ver[1,16]" "def[1]"    "def[2]"   
[67] "def[3]"    "def[4]"    "def[5]"    "def[6]"    "def[7]"    "def[8]"   
[73] "def[9]"    "def[10]"   "def[11]"   "def[12]"   "def[13]"   "def[14]"  

## Tamaño del problema

[Contenido](#Contenido)

In [109]:
N = length(var.names)

In [110]:
m <- make.lp(nrow = 0, ncol = N) 

In [1]:
w <- rep(0, N)
names(w) <- var.names

ERROR: Error in eval(expr, envir, enclos): object 'N' not found


## Función objetivo

[Contenido](#Contenido)

En esta porción de código se ejemplifica la construcción de la función objetivo usando las tablas definidas anteriormente. Recuerde que se desea minimizar el costo total del racionamiento más el costo total de la generación térmica:

$$\text{minimize} ~ \sum_{p=1}^{P} ~\Bigg\{ CR * DEF_p + \sum_{t=1}^T CC_t * GT_{p,t} ~\Bigg\}$$

In [95]:
w[] = 0  ## hace cero todos los coeficientes

## deficit
for(i in var.def) w[i] <- CR

## generación termica
for(t in 1:T)
    w[var.gt[t,]] <- termos$CC[t]  

printvec(w)

15 * gt[1,1]
15 * gt[1,2]
15 * gt[1,3]
15 * gt[1,4]
15 * gt[1,5]
15 * gt[1,6]
15 * gt[1,7]
15 * gt[1,8]
15 * gt[1,9]
15 * gt[1,10]
15 * gt[1,11]
15 * gt[1,12]
15 * gt[1,13]
15 * gt[1,14]
15 * gt[1,15]
15 * gt[1,16]
1000 * def[1]
1000 * def[2]
1000 * def[3]
1000 * def[4]
1000 * def[5]
1000 * def[6]
1000 * def[7]
1000 * def[8]
1000 * def[9]
1000 * def[10]
1000 * def[11]
1000 * def[12]
1000 * def[13]
1000 * def[14]
1000 * def[15]
1000 * def[16]


In [96]:
## función objetivo
set.objfn(m, w)

## Restricciones de demanda

[Contenido](#Contenido)

Por cada periodo de tiempo $p$ debe cumplirse que:

$$ DEF_p + \Bigg\{ \sum_{h=1}^H \rho_h * Q_{hp} \Bigg\} + \Bigg\{ \sum_{t=1}^T GT_{t p} \Bigg\} = dem_p$$

In [97]:
for(p in 1:P) {

    w[] = 0  ## hace cero todos los coeficientes

    ## deficit
    w[var.def[p]] <- 1
 
    ## generación hidraulica
    for (h in 1:H) 
        w[var.q[h,p]] <- hidros$rho[h] 
    
    ## generación termica
    w[var.gt[1:T, p]] <- 1 
    
    add.constraint(m, w, "=", dem[p])

    printvec(w)
    cat("=\n")
    cat(dem[p])
    cat('\n')
    cat('\n')    
        
}

1 * gt[1,1]
1 * q[1,1]
1 * def[1]
=
50

1 * gt[1,2]
1 * q[1,2]
1 * def[2]
=
50

1 * gt[1,3]
1 * q[1,3]
1 * def[3]
=
50

1 * gt[1,4]
1 * q[1,4]
1 * def[4]
=
50

1 * gt[1,5]
1 * q[1,5]
1 * def[5]
=
60

1 * gt[1,6]
1 * q[1,6]
1 * def[6]
=
60

1 * gt[1,7]
1 * q[1,7]
1 * def[7]
=
60

1 * gt[1,8]
1 * q[1,8]
1 * def[8]
=
60

1 * gt[1,9]
1 * q[1,9]
1 * def[9]
=
70

1 * gt[1,10]
1 * q[1,10]
1 * def[10]
=
70

1 * gt[1,11]
1 * q[1,11]
1 * def[11]
=
70

1 * gt[1,12]
1 * q[1,12]
1 * def[12]
=
70

1 * gt[1,13]
1 * q[1,13]
1 * def[13]
=
80

1 * gt[1,14]
1 * q[1,14]
1 * def[14]
=
80

1 * gt[1,15]
1 * q[1,15]
1 * def[15]
=
80

1 * gt[1,16]
1 * q[1,16]
1 * def[16]
=
80



## Continuidad de los volumenes de los embalses

[Contenido](#Contenido)

Para el primer periodo, el embalse inicial $Vol_{h,0}$ es conocido, entonces:

$$Vol_{h,1} +Q_{h,1}+Ver_{h,1}=A_{h,1}+ Vol_{h,0}$$

In [98]:
## para el periodo 1 y para cada planta hidraulica
for (h in 1:H) {

    w[] = 0  ## hace cero todos los coeficientes
    
    w[var.vol[h,1]]  <- 1
    w[var.q[h,1]]    <- 1
    w[var.ver[h,1]]  <- 1
    
    rhs <- aportes[aportes$planta == h & aportes$t == 1, 'aportes'] + hidros$Vo[h]
    add.constraint(m, w, "=", rhs)

    printvec(w)
    cat('=\n')
    cat(rhs)
    cat('\n')
    cat('\n')
}


1 * vol[1,1]
1 * q[1,1]
1 * ver[1,1]
=
98



Para cada planta hidráulica $h$ y cada periodo de tiempo $p$ se debe cumplir que:

$$Vol_{h,p} - Vol_{h,p-1}+Q_{h,p}+Ver_{h,p}=A_{h,p}$$

In [99]:
## para el periodo 2 y siguientes
for(p in 2:P) {
    
    for (h in 1:H) {
    
        w[] = 0  ## hace cero todos los coeficientes
        
        w[var.vol[h,p]]    <-  1
        w[var.vol[h,p-1]]  <- -1
        w[var.q[h,p]]      <-  1
        w[var.ver[h,p]]    <-  1
        
        rhs <- aportes[aportes$planta == h & aportes$t == p, 'aportes']
        
        add.constraint(m, w, "=", rhs)
    
        printvec(w)
        cat('=\n')
        cat(rhs)
        cat('\n')
        cat('\n')
    }    
}

-1 * vol[1,1]
1 * vol[1,2]
1 * q[1,2]
1 * ver[1,2]
=
22

-1 * vol[1,2]
1 * vol[1,3]
1 * q[1,3]
1 * ver[1,3]
=
19

-1 * vol[1,3]
1 * vol[1,4]
1 * q[1,4]
1 * ver[1,4]
=
25

-1 * vol[1,4]
1 * vol[1,5]
1 * q[1,5]
1 * ver[1,5]
=
21

-1 * vol[1,5]
1 * vol[1,6]
1 * q[1,6]
1 * ver[1,6]
=
16

-1 * vol[1,6]
1 * vol[1,7]
1 * q[1,7]
1 * ver[1,7]
=
18

-1 * vol[1,7]
1 * vol[1,8]
1 * q[1,8]
1 * ver[1,8]
=
23

-1 * vol[1,8]
1 * vol[1,9]
1 * q[1,9]
1 * ver[1,9]
=
27

-1 * vol[1,9]
1 * vol[1,10]
1 * q[1,10]
1 * ver[1,10]
=
22

-1 * vol[1,10]
1 * vol[1,11]
1 * q[1,11]
1 * ver[1,11]
=
23

-1 * vol[1,11]
1 * vol[1,12]
1 * q[1,12]
1 * ver[1,12]
=
33

-1 * vol[1,12]
1 * vol[1,13]
1 * q[1,13]
1 * ver[1,13]
=
26

-1 * vol[1,13]
1 * vol[1,14]
1 * q[1,14]
1 * ver[1,14]
=
19

-1 * vol[1,14]
1 * vol[1,15]
1 * q[1,15]
1 * ver[1,15]
=
22

-1 * vol[1,15]
1 * vol[1,16]
1 * q[1,16]
1 * ver[1,16]
=
27



Para el último periodo, el embalse final es conocido.

In [100]:
## para el periodo P
for (h in 1:H) {

    w[] = 0  ## hace cero todos los coeficientes

    w[var.vol[h,P]] <- 1
    rhs = hidros$Vf[h]
    add.constraint(m, w, "=", rhs)
    
    printvec(w)
    cat('=\n')
    cat(rhs)
    cat('\n')
    cat('\n')

}

1 * vol[1,16]
=
70



## Generación máxima

[Contenido](#Contenido)

Para cada planta hidraulica $h$ y por cada periodo de tiempo $p$:

$$Q_{h,p} <= Qmax_h$$

In [101]:
for(p in 1:P) {
    for (h in 1:H) {

        w[] = 0  ## hace cero todos los coeficientes

        w[var.q[h, p]]  <- 1
        rhs <- hidros$Qmax[h]
        add.constraint(m, w, "<=", rhs)
        
        printvec(w)
        cat('<=\n')
        cat(rhs)
        cat('\n')
        cat('\n')
    }
}

1 * q[1,1]
<=
30

1 * q[1,2]
<=
30

1 * q[1,3]
<=
30

1 * q[1,4]
<=
30

1 * q[1,5]
<=
30

1 * q[1,6]
<=
30

1 * q[1,7]
<=
30

1 * q[1,8]
<=
30

1 * q[1,9]
<=
30

1 * q[1,10]
<=
30

1 * q[1,11]
<=
30

1 * q[1,12]
<=
30

1 * q[1,13]
<=
30

1 * q[1,14]
<=
30

1 * q[1,15]
<=
30

1 * q[1,16]
<=
30



Por cada planta térmica $t$ y por cada periodo de tiempo $p$:

$$GT_{t,p} <= GTmax_{t}$$

In [102]:
for(p in 1:P) {
    for (t in 1:T) {

        w[] = 0  ## hace cero todos los coeficientes

        w[var.gt[t,p]] <- 1
        rhs <- termos$GTmax[t]
        
        add.constraint(m, w, "<=", rhs)
        
        printvec(w)
        cat('<=\n')
        cat(rhs)
        cat('\n')
        cat('\n')
    }
}

1 * gt[1,1]
<=
31

1 * gt[1,2]
<=
31

1 * gt[1,3]
<=
31

1 * gt[1,4]
<=
31

1 * gt[1,5]
<=
31

1 * gt[1,6]
<=
31

1 * gt[1,7]
<=
31

1 * gt[1,8]
<=
31

1 * gt[1,9]
<=
31

1 * gt[1,10]
<=
31

1 * gt[1,11]
<=
31

1 * gt[1,12]
<=
31

1 * gt[1,13]
<=
31

1 * gt[1,14]
<=
31

1 * gt[1,15]
<=
31

1 * gt[1,16]
<=
31



## Volumen máximo

[Contenido](#Contenido)

In [103]:
for(p in 1:P) {
    for (h in 1:H) {

        w[] = 0  ## hace cero todos los coeficientes
        
        w[var.vol[h,p]] <- 1
        rhs = hidros$Vmax[h]
        add.constraint(m, w, "<=", rhs)
        
        printvec(w)
        cat('<=\n')
        cat(rhs)
        cat('\n')
        cat('\n')
    }
}

1 * vol[1,1]
<=
100

1 * vol[1,2]
<=
100

1 * vol[1,3]
<=
100

1 * vol[1,4]
<=
100

1 * vol[1,5]
<=
100

1 * vol[1,6]
<=
100

1 * vol[1,7]
<=
100

1 * vol[1,8]
<=
100

1 * vol[1,9]
<=
100

1 * vol[1,10]
<=
100

1 * vol[1,11]
<=
100

1 * vol[1,12]
<=
100

1 * vol[1,13]
<=
100

1 * vol[1,14]
<=
100

1 * vol[1,15]
<=
100

1 * vol[1,16]
<=
100



# Solucion

[Contenido](#Contenido)

In [104]:
m

Model name: 
  a linear program with 80 decision variables and 81 constraints

In [105]:
#   0:   "optimal solution found"
#   1:   "the model is sub-optimal"
#   2:   "the model is infeasible"
solve(m)

[1] 0

In [106]:
w[] <- get.variables(m)

In [107]:
## generacion termica
w[var.gt]

gt[1,1]  gt[1,2]  gt[1,3]  gt[1,4]  gt[1,5]  gt[1,6]  gt[1,7]  gt[1,8] 
      31       31       31       31       31       31       31       31 
 gt[1,9] gt[1,10] gt[1,11] gt[1,12] gt[1,13] gt[1,14] gt[1,15] gt[1,16] 
      31       31       31       31       31       31       31       31

In [108]:
## deficit
w[var.def]

def[1]  def[2]  def[3]  def[4]  def[5]  def[6]  def[7]  def[8]  def[9] def[10] 
      0       0       0       0       0       0       0       0      10       9 
def[11] def[12] def[13] def[14] def[15] def[16] 
     39       9      49      19      19      19

In [111]:
## volumen del embalse
w[var.vol]

vol[1,1]  vol[1,2]  vol[1,3]  vol[1,4]  vol[1,5]  vol[1,6]  vol[1,7]  vol[1,8] 
       79        82        82        88        80        67        56        50 
 vol[1,9] vol[1,10] vol[1,11] vol[1,12] vol[1,13] vol[1,14] vol[1,15] vol[1,16] 
       48        40        63        66        92        81        73        70

---

[Contenido](#Contenido)

<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Licencia de Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br />Este obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">licencia de Creative Commons Reconocimiento-NoComercial-SinObraDerivada 4.0 Internacional</a>.